In [1]:
import sys
import os
if not any(path.endswith('textbook') for path in sys.path):
    sys.path.append(os.path.abspath('../../..'))
from textbook_utils import *

In [4]:
df = pd.read_csv('data/fake_news.csv', parse_dates=['timestamp'])

(sec:fake_news_exploring)=
# Exploring the Data